In [1]:
%config IPCompleter.use_jedi = False
# %xmode Verbose
# %xmode context
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
# from numba import jit
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
# pd.options.mode.dtype_backend = 'pyarrow' # use new pyarrow backend instead of numpy
from attrs import define, field, fields, Factory
import tables as tb
from datetime import datetime, timedelta

# Pho's Formatting Preferences
import builtins

import IPython
from IPython.core.formatters import PlainTextFormatter
from IPython import get_ipython

from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
set_pho_preferences_concise()
# Jupyter-lab enable printing for any line on its own (instead of just the last one in the cell)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# BEGIN PPRINT CUSTOMIZATION ___________________________________________________________________________________________ #


## IPython pprint
from pyphocorehelpers.pprint import wide_pprint, wide_pprint_ipython, wide_pprint_jupyter, MAX_LINE_LENGTH

# Override default pprint
builtins.pprint = wide_pprint

text_formatter: PlainTextFormatter = IPython.get_ipython().display_formatter.formatters['text/plain']
text_formatter.max_width = MAX_LINE_LENGTH
text_formatter.for_type(object, wide_pprint_jupyter)


# END PPRINT CUSTOMIZATION ___________________________________________________________________________________________ #

from pyphocorehelpers.print_helpers import get_now_time_str, get_now_day_str

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path, file_uri_from_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from typing_extensions import TypeAlias
from nptyping import NDArray
import neuropy.utils.type_aliases as types

from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange, Epoch
from neuropy.core.ratemap import Ratemap
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.utils.matplotlib_helpers import matplotlib_file_only, matplotlib_configuration, matplotlib_configuration_update
from neuropy.core.neuron_identities import NeuronIdentityTable, neuronTypesList, neuronTypesEnum
from neuropy.utils.mixins.AttrsClassHelpers import AttrsBasedClassHelperMixin, serialized_field, serialized_attribute_field, non_serialized_field, custom_define
from neuropy.utils.mixins.HDF5_representable import HDF_DeserializationMixin, post_deserialize, HDF_SerializationMixin, HDFMixin, HDF_Converter

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core.neurons import NeuronType
from neuropy.core.user_annotations import UserAnnotationsManager
from neuropy.core.position import Position
from neuropy.core.session.dataSession import DataSession
from neuropy.analyses.time_dependent_placefields import PfND_TimeDependent, PlacefieldSnapshot
from neuropy.utils.debug_helpers import debug_print_placefield, debug_print_subsession_neuron_differences, debug_print_ratemap, debug_print_spike_counts, debug_plot_2d_binning, print_aligned_columns
from neuropy.utils.debug_helpers import parameter_sweeps, _plot_parameter_sweep, compare_placefields_info
from neuropy.utils.indexing_helpers import NumpyHelpers, union_of_arrays, intersection_of_arrays, find_desired_sort_indicies, paired_incremental_sorting
from pyphocorehelpers.print_helpers import print_object_memory_usage, print_dataframe_memory_usage, print_value_overview_only, DocumentationFilePrinter, print_keys_if_possible, generate_html_string, document_active_variables

## Pho Programming Helpers:
import inspect
from pyphocorehelpers.print_helpers import DocumentationFilePrinter, TypePrintMode, print_keys_if_possible, debug_dump_object_member_shapes, print_value_overview_only, document_active_variables
from pyphocorehelpers.programming_helpers import IPythonHelpers, PythonDictionaryDefinitionFormat, MemoryManagement, inspect_callable_arguments, get_arguments_as_optional_dict, GeneratedClassDefinitionType, CodeConversion
from pyphocorehelpers.gui.Qt.TopLevelWindowHelper import TopLevelWindowHelper, print_widget_hierarchy
from pyphocorehelpers.indexing_helpers import reorder_columns, reorder_columns_relative, dict_to_full_array
# doc_output_parent_folder: Path = Path('EXTERNAL/DEVELOPER_NOTES/DataStructureDocumentation').resolve() # ../.
# print(f"doc_output_parent_folder: {doc_output_parent_folder}")
# assert doc_output_parent_folder.exists()

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_load_session, batch_extended_computations, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme

import pyphoplacecellanalysis.External.pyqtgraph as pg

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons
from pyphoplacecellanalysis.General.Batch.runBatch import BatchSessionCompletionHandler # for `post_compute_validate(...)`
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BasePositionDecoder
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends # for `_perform_long_short_instantaneous_spike_rate_groups_analysis`
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import SingleBarResult, InstantaneousSpikeRateGroupsComputation, TruncationCheckingResults # for `BatchSessionCompletionHandler`, `AcrossSessionsAggregator`
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalPlacefieldGlobalComputationFunctions, DirectionalLapsResult, TrackTemplates
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderGlobalComputationFunctions
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import TrackTemplates
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderComputationsContainer, RankOrderResult
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderAnalyses


# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()

matplotlib.use('Qt5Agg')
# %matplotlib inline
# %matplotlib auto

# _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
_restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

# import pylustrator # call `pylustrator.start()` before creating your first figure in code.
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap_pyqtgraph # used in `plot_kourosh_activity_style_figure`
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot, plot_raster_plot
from pyphoplacecellanalysis.General.Mixins.DataSeriesColorHelpers import UnitColoringMode, DataSeriesColorHelpers
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import _build_default_tick, build_scatter_plot_kwargs
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.Render2DScrollWindowPlot import Render2DScrollWindowPlotMixin, ScatterItemData
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_extended_programmatic_figures, batch_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends
from pyphoplacecellanalysis.General.Mixins.SpikesRenderingBaseMixin import SpikeEmphasisState

from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PAPER_FIGURE_figure_1_add_replay_epoch_rasters, PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_3, main_complete_figure_generations
from pyphoplacecellanalysis.SpecificResults.fourthYearPresentation import *

# Jupyter Widget Interactive
import ipywidgets as widgets
from IPython.display import display, HTML
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager
from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import interactive_pipeline_widget, interactive_pipeline_files
from pyphocorehelpers.gui.Jupyter.simple_widgets import fullwidth_path_widget, render_colors

from datetime import datetime, date, timedelta
from pyphocorehelpers.print_helpers import get_now_day_str, get_now_rounded_time_str

DAY_DATE_STR: str = date.today().strftime("%Y-%m-%d")
DAY_DATE_TO_USE = f'{DAY_DATE_STR}' # used for filenames throught the notebook
print(f'DAY_DATE_STR: {DAY_DATE_STR}, DAY_DATE_TO_USE: {DAY_DATE_TO_USE}')

NOW_DATETIME: str = get_now_rounded_time_str()
NOW_DATETIME_TO_USE = f'{NOW_DATETIME}' # used for filenames throught the notebook
print(f'NOW_DATETIME: {NOW_DATETIME}, NOW_DATETIME_TO_USE: {NOW_DATETIME_TO_USE}')


from pyphocorehelpers.gui.Jupyter.simple_widgets import build_global_data_root_parent_path_selection_widget
all_paths = [Path('/Volumes/SwapSSD/Data'), Path('/Users/pho/data'), Path(r'/media/MAX/Data'), Path(r'/media/halechr/MAX/Data'), Path(r'/home/halechr/FastData'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')]
global_data_root_parent_path = None
def on_user_update_path_selection(new_path: Path):
    global global_data_root_parent_path
    new_global_data_root_parent_path = new_path.resolve()
    global_data_root_parent_path = new_global_data_root_parent_path
    print(f'global_data_root_parent_path changed to {global_data_root_parent_path}')
    assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
            
global_data_root_parent_path_widget = build_global_data_root_parent_path_selection_widget(all_paths, on_user_update_path_selection)
global_data_root_parent_path_widget

from pyphoplacecellanalysis.Analysis.Decoder.heuristic_replay_scoring import HeuristicReplayScoring, compute_local_peak_probabilities, get_peaks_mask, expand_peaks_mask, InversionCount, is_valid_sequence_index, _compute_sequences_spanning_ignored_intrusions
from pyphoplacecellanalysis.Analysis.Decoder.heuristic_replay_scoring import _compute_diffusion_value

Automatic pdb calling has been turned OFF
DAY_DATE_STR: 2024-05-09, DAY_DATE_TO_USE: 2024-05-09
NOW_DATETIME: 2024-05-09_0635AM, NOW_DATETIME_TO_USE: 2024-05-09_0635AM
global_data_root_parent_path changed to W:\Data


In [2]:
import pyphoplacecellanalysis.External.pyqtgraph as pg
from pyphoplacecellanalysis.External.pyqtgraph.Qt import QtGui, QtCore, QtWidgets
# from pyphoplacecellanalysis.External.pyqtgraph.parametertree.parameterTypes.file import popupFilePicker
from pyphoplacecellanalysis.External.pyqtgraph.widgets.FileDialog import FileDialog

from silx.gui import qt
from silx.gui.dialog.ImageFileDialog import ImageFileDialog
from silx.gui.dialog.DataFileDialog import DataFileDialog
import silx.io

from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import saveFile

from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import loadData

from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import DecodedFilterEpochsResult
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DecoderDecodedEpochsResult


# load_path = Path(r"W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\output\2024-02-13_CustomDecodingResults.pkl").resolve()
# load_path = Path(r"W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\output\2024-02-13_9pm_CustomDecodingResults.pkl").resolve()
# load_path = Path(r"W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\output\2024-02-14_CustomDecodingResults.pkl").resolve()
# load_path = Path(r"W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\output\2024-02-15_CustomDecodingResults.pkl").resolve()
# load_path = Path("/media/halechr/MAX/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/2024-02-15_CustomDecodingResults.pkl").resolve()
# load_path = Path("/Users/pho/data/KDIBA/gor01/one/2006-6-09_1-22-43/output/2024-02-16_CustomDecodingResults.pkl").resolve()
# load_path = Path("/Users/pho/data/KDIBA/gor01/one/2006-6-09_1-22-43/output/2024-02-14_CustomDecodingResults.pkl").resolve()
# load_path = Path("/Users/pho/data/KDIBA/gor01/one/2006-6-09_1-22-43/output/2024-04-25_CustomDecodingResults.pkl").resolve()
# load_path = Path(r"W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\output\2024-04-25_CustomDecodingResults.pkl").resolve() # APOGEE
# load_path = Path(r"W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\output\2024-04-28_CustomDecodingResults.pkl").resolve()
# load_path = Path(r"W:\Data\KDIBA\gor01\two\2006-6-09_22-24-40\output\2024-04-28_CustomDecodingResults.pkl").resolve() # APOGEE
load_path = Path(r"W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\output\2024-04-28_CustomDecodingResults.pkl").resolve()
# load_path = Path(r"W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\output\2024-05-06_CustomDecodingResults.pkl").resolve() # new relative format

assert load_path.exists()
# loaded_dict = loadData(load_path, debug_print=False)
# print_keys_if_possible('loaded_dict', loaded_dict)

base_loaded_dict = loadData(load_path, debug_print=False)
xbin = base_loaded_dict.pop('xbin', None)
xbin_centers = base_loaded_dict.pop('xbin_centers', None)
# ybin = deepcopy(long_pf2D.ybin)
# ybin_centers = deepcopy(long_pf2D.ybin_centers)
print(f"xbin_centers: {xbin_centers}")

loaded_dict = base_loaded_dict['directional_decoders_epochs_decode_result']


## UNPACK HERE:
pos_bin_size: float = loaded_dict['pos_bin_size'] # 3.8632841399651463
ripple_decoding_time_bin_size = loaded_dict['ripple_decoding_time_bin_size']
laps_decoding_time_bin_size = loaded_dict['laps_decoding_time_bin_size']
decoder_laps_filter_epochs_decoder_result_dict = loaded_dict['decoder_laps_filter_epochs_decoder_result_dict']
decoder_ripple_filter_epochs_decoder_result_dict = loaded_dict['decoder_ripple_filter_epochs_decoder_result_dict']
decoder_laps_radon_transform_df_dict = loaded_dict['decoder_laps_radon_transform_df_dict']
decoder_ripple_radon_transform_df_dict = loaded_dict['decoder_ripple_radon_transform_df_dict']
## New 2024-02-14 - Noon:
decoder_laps_radon_transform_extras_dict = loaded_dict['decoder_laps_radon_transform_extras_dict']
decoder_ripple_radon_transform_extras_dict = loaded_dict['decoder_ripple_radon_transform_extras_dict']

laps_weighted_corr_merged_df = loaded_dict['laps_weighted_corr_merged_df']
ripple_weighted_corr_merged_df = loaded_dict['ripple_weighted_corr_merged_df']
laps_simple_pf_pearson_merged_df = loaded_dict['laps_simple_pf_pearson_merged_df']
ripple_simple_pf_pearson_merged_df = loaded_dict['ripple_simple_pf_pearson_merged_df']

_VersionedResultMixin_version = loaded_dict.pop('_VersionedResultMixin_version', None)

# ## saved with:
# save_dict = {
# 'directional_decoders_epochs_decode_result': directional_decoders_epochs_decode_result.__getstate__(),
# 'xbin': xbin, 'xbin_centers': xbin_centers
# }

directional_decoders_epochs_decode_result: DecoderDecodedEpochsResult = DecoderDecodedEpochsResult(**loaded_dict)
# {'ripple_decoding_time_bin_size':ripple_decoding_time_bin_size, 'laps_decoding_time_bin_size':laps_decoding_time_bin_size, 'decoder_laps_filter_epochs_decoder_result_dict':decoder_laps_filter_epochs_decoder_result_dict, 'decoder_ripple_filter_epochs_decoder_result_dict':decoder_ripple_filter_epochs_decoder_result_dict, 'decoder_laps_radon_transform_df_dict':decoder_laps_radon_transform_df_dict, 'decoder_ripple_radon_transform_df_dict':decoder_ripple_radon_transform_df_dict}

# pos_bin_size


epoch_type_selector_dict = {'laps': {'decoder_filter_epochs_decoder_result_dict': decoder_laps_filter_epochs_decoder_result_dict, #'decoder_laps_radon_transform_df_dict': decoder_laps_radon_transform_df_dict,
          'decoder_radon_transform_extras_dict': decoder_laps_radon_transform_extras_dict},
'ripple': {'decoder_filter_epochs_decoder_result_dict': decoder_ripple_filter_epochs_decoder_result_dict, #'decoder_laps_radon_transform_df_dict': decoder_laps_radon_transform_df_dict,
          'decoder_radon_transform_extras_dict': decoder_ripple_radon_transform_extras_dict},
}


Loading loaded session pickle file results : W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\output\2024-04-28_CustomDecodingResults.pkl... done.
xbin_centers: [38.9801 42.7855 46.5909 50.3963 54.2018 58.0072 61.8126 65.618 69.4234 73.2289 77.0343 80.8397 84.6451 88.4505 92.2559 96.0614 99.8668 103.672 107.478 111.283 115.088 118.894 122.699 126.505 130.31 134.116 137.921 141.726 145.532 149.337 153.143 156.948 160.753 164.559 168.364 172.17 175.975 179.781 183.586 187.391 191.197 195.002 198.808 202.613 206.418 210.224 214.029 217.835 221.64 225.446 229.251 233.056 236.862 240.667 244.473 248.278 252.083]


In [3]:
# a_decoder_decoded_epochs_result: DecodedFilterEpochsResult = deepcopy(directional_decoders_epochs_decode_result.decoder_laps_filter_epochs_decoder_result_dict['long_LR'])
a_decoder_decoded_epochs_result: DecodedFilterEpochsResult = deepcopy(directional_decoders_epochs_decode_result.decoder_ripple_filter_epochs_decoder_result_dict['long_LR'])
a_decoder_decoded_epochs_result

DecodedFilterEpochsResult(decoding_time_bin_size: float,
	filter_epochs: pandas.core.frame.DataFrame,
	num_filter_epochs: int,
	most_likely_positions_list: list | shape (n_epochs),
	p_x_given_n_list: list | shape (n_epochs),
	marginal_x_list: list | shape (n_epochs),
	marginal_y_list: list | shape (n_epochs),
	most_likely_position_indicies_list: list | shape (n_epochs),
	spkcount: list | shape (n_epochs),
	nbins: numpy.ndarray | shape (n_epochs),
	time_bin_containers: list | shape (n_epochs),
	time_bin_edges: list | shape (n_epochs),
	epoch_description_list: list | shape (n_epochs)
)

In [4]:
good_epochs_list = [12, 15, 28, 37]


In [ ]:

app = pg.mkQApp('Heuristic Analysis Explorer')


## 2024-05-03 Heuristic 

In [5]:
import numpy.ma as ma # used in `most_likely_directional_rank_order_shuffling`
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import SingleEpochDecodedResult
from PIL import Image
from pyphocorehelpers.plotting.media_output_helpers import get_array_as_image
from scipy.signal import convolve2d

active_epoch_idx: int = good_epochs_list[0]
print(f'active_epoch_idx: {active_epoch_idx}')

time_bin_size: float = a_decoder_decoded_epochs_result.decoding_time_bin_size
active_captured_single_epoch_result: SingleEpochDecodedResult = a_decoder_decoded_epochs_result.get_result_for_epoch(active_epoch_idx=active_epoch_idx)

time_bin_centers = deepcopy(active_captured_single_epoch_result.time_bin_container.centers)
t_start, t_end = active_captured_single_epoch_result.time_bin_edges[0], active_captured_single_epoch_result.time_bin_edges[-1]

p_x_given_n = deepcopy(active_captured_single_epoch_result.p_x_given_n)
most_likely_positions = deepcopy(active_captured_single_epoch_result.most_likely_positions)
most_likely_positionIndicies = deepcopy(active_captured_single_epoch_result.most_likely_position_indicies)

# HeuristicReplayScoring.bin_wise_jump_distance(a_result=a_decoder_decoded_epochs_result, an_epoch_idx=active_epoch_idx, a_decoder_track_length=170.0)

HeuristicReplayScoring.bin_wise_track_coverage_score_fn(a_result=a_decoder_decoded_epochs_result, an_epoch_idx=active_epoch_idx, a_decoder_track_length=170.0)

## Convert from a probability matrix to a cost matrix by computing (1.0 - P), so the most probable have the lowest values
costs_matrix = 1.0 - deepcopy(p_x_given_n)
# costs_matrix

uniform_diffusion_prob: float = _compute_diffusion_value(p_x_given_n) # single bin diffusion probability
print(f'uniform_diffusion_prob: {uniform_diffusion_prob}')
is_higher_than_diffusion = (p_x_given_n > uniform_diffusion_prob)

p_x_given_n_masked = ma.masked_array(p_x_given_n, mask=np.logical_not(is_higher_than_diffusion), fill_value=np.nan)
# p_x_given_n_masked


active_epoch_idx: 12


0.2982456140350877

uniform_diffusion_prob: 0.017543859649122806


In [6]:
def _get_epoch_posterior(active_epoch_idx: int):
    print(f'active_epoch_idx: {active_epoch_idx}')

    time_bin_size: float = a_decoder_decoded_epochs_result.decoding_time_bin_size
    active_captured_single_epoch_result: SingleEpochDecodedResult = a_decoder_decoded_epochs_result.get_result_for_epoch(active_epoch_idx=active_epoch_idx)

    time_bin_centers = deepcopy(active_captured_single_epoch_result.time_bin_container.centers)
    t_start, t_end = active_captured_single_epoch_result.time_bin_edges[0], active_captured_single_epoch_result.time_bin_edges[-1]

    p_x_given_n = deepcopy(active_captured_single_epoch_result.p_x_given_n)
    most_likely_positions = deepcopy(active_captured_single_epoch_result.most_likely_positions)
    most_likely_positionIndicies = deepcopy(active_captured_single_epoch_result.most_likely_position_indicies)

    ## Convert from a probability matrix to a cost matrix by computing (1.0 - P), so the most probable have the lowest values
    costs_matrix = 1.0 - deepcopy(p_x_given_n)
    # costs_matrix

    uniform_diffusion_prob: float = _compute_diffusion_value(p_x_given_n) # single bin diffusion probability
    print(f'uniform_diffusion_prob: {uniform_diffusion_prob}')
    is_higher_than_diffusion = (p_x_given_n > uniform_diffusion_prob)

    p_x_given_n_masked = ma.masked_array(p_x_given_n, mask=np.logical_not(is_higher_than_diffusion), fill_value=np.nan)
    return p_x_given_n_masked


_get_epoch_posterior(active_epoch_idx=good_epochs_list[0])

active_epoch_idx: 12
uniform_diffusion_prob: 0.017543859649122806


masked_array(
  data=[[--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, --],
        [--, --, --, --, --, 0.026905143371692662],
        [--, --,

In [7]:
import numpy as np
from attrs import astuple, asdict # used in `UnpackableMixin`
from silx.gui.plot import Plot2D
from silx.gui.colors import Colormap
from silx.gui.plot.items import ImageBase
from pyphoplacecellanalysis.Analysis.Decoder.heuristic_replay_scoring import HeuristicScoresTuple

## Uses: xbin, t_start, pos_bin_size, time_bin_size

@define(slots=False)
class EpochHeuristicDebugger:
    active_single_epoch_result: SingleEpochDecodedResult = field(default=None)
    p_x_given_n_masked: NDArray = field(default=None) # deepcopy(p_x_given_n_masked) # .T
    heuristic_scores: HeuristicScoresTuple = field(default=None)

    # a_cmap = Colormap(name="viridis", vmin=0, vmax=1)
    a_cmap: Colormap = field(factory=(lambda *args, **kwargs: Colormap(name="viridis", vmin=0))) # , vmax=1    
    plot: Plot2D = field(factory=Plot2D)

    def build_ui(self):
        img_origin = (0.0, 0.0)
        # img_origin = (t_start, xbin[0]) # (origin X, origin Y)
        # img_origin = (xbin[0], t_start) # (origin X, origin Y)
        img_scale = (1.0, 1.0)
        # img_scale = ((1.0/(t_end - t_start)), (1.0/(xbin[-1] - xbin[0])))
        # img_scale = (pos_bin_size, time_bin_size) # ??
        # img_scale = (1.0/float(pos_bin_size), 1.0/float(time_bin_size))
        # 

        print(f'img_origin: {img_origin}')
        print(f'img_scale: {img_scale}')

        # label_kwargs = dict(xlabel='t (sec)', ylabel='x (cm)')
        label_kwargs = dict(xlabel='t (bin)', ylabel='x (bin)')


        self.plot.addImage(self.p_x_given_n_masked, legend='p_x_given_n', replace=True, colormap=self.a_cmap, origin=img_origin, scale=img_scale, **label_kwargs,
                    resetzoom=True) # , colormap="viridis", vmin=0, vmax=1
        prev_img: ImageBase = self.plot.getImage('p_x_given_n')

        self.plot.show()


    def on_slider_change(self, change):
        ### Updates the active epoch via a slider:

        # print("Slider value:", change.new)
        active_epoch_idx: int = int(change.new)
        print(f'epoch[{active_epoch_idx}]')

        # Data Update ________________________________________________________________________________________________________ #
        # active_captured_single_epoch_result = a_decoder_decoded_epochs_result.get_result_for_epoch(active_epoch_idx=active_epoch_idx) 
        # self.p_x_given_n_masked = _get_epoch_posterior(active_epoch_idx=active_epoch_idx)

        time_bin_size: float = a_decoder_decoded_epochs_result.decoding_time_bin_size
        self.active_single_epoch_result: SingleEpochDecodedResult = a_decoder_decoded_epochs_result.get_result_for_epoch(active_epoch_idx=active_epoch_idx)

        time_bin_centers = deepcopy(self.active_single_epoch_result.time_bin_container.centers)
        t_start, t_end = self.active_single_epoch_result.time_bin_edges[0], self.active_single_epoch_result.time_bin_edges[-1]

        p_x_given_n = deepcopy(self.active_single_epoch_result.p_x_given_n)
        most_likely_positions = deepcopy(self.active_single_epoch_result.most_likely_positions)
        most_likely_positionIndicies = deepcopy(self.active_single_epoch_result.most_likely_position_indicies)

        ## Convert from a probability matrix to a cost matrix by computing (1.0 - P), so the most probable have the lowest values
        costs_matrix = 1.0 - deepcopy(p_x_given_n)
        # costs_matrix
        uniform_diffusion_prob: float = _compute_diffusion_value(p_x_given_n) # single bin diffusion probability
        print(f'uniform_diffusion_prob: {uniform_diffusion_prob}')
        is_higher_than_diffusion = (p_x_given_n > uniform_diffusion_prob)

        self.p_x_given_n_masked = ma.masked_array(p_x_given_n, mask=np.logical_not(is_higher_than_diffusion), fill_value=np.nan)
        

        self.heuristic_scores = HeuristicReplayScoring.compute_pho_heuristic_replay_scores(a_result=a_decoder_decoded_epochs_result, an_epoch_idx=self.active_single_epoch_result.epoch_data_index, debug_print=False)
        # longest_sequence_length, longest_sequence_length_ratio, direction_change_bin_ratio, congruent_dir_bins_ratio, total_congruent_direction_change, total_variation, integral_second_derivative, stddev_of_diff, position_derivatives_df = self.heuristic_scores
        # np.diff(active_captured_single_epoch_result.most_likely_position_indicies)
        # print(f'heuristic_scores: {astuple(self.heuristic_scores)[:-1]}')
        print(f"heuristic_scores: {asdict(self.heuristic_scores, filter=(lambda an_attr, attr_value: an_attr.name not in ['position_derivatives_df']))}")

        # Plottings __________________________________________________________________________________________________________ #
        prev_img: ImageBase = self.plot.getImage('p_x_given_n')
        prev_img.setData(self.p_x_given_n_masked)
        # prev_img._setYLabel(f'epoch[{active_epoch_idx}: x (bin)')

        max_path = np.nanargmax(self.p_x_given_n_masked, axis=0) # returns the x-bins that maximize the path
        assert len(max_path) == len(time_bin_centers)
        # _curve_x = time_bin_centers
        _curve_x = np.arange(len(max_path)) + 0.5 # move forward by a half bin

        # a_track_length: float = 170.0
        # effectively_same_location_size = 0.1 * a_track_length # 10% of the track length
        # effectively_same_location_num_bins: int = np.rint(effectively_same_location_size)
        effectively_same_location_num_bins: int = 4
        _max_path_Curve = self.plot.addCurve(x=_curve_x, y=max_path, color='r', symbol='s', legend='max_path', replace=True, yerror=effectively_same_location_num_bins)
        # _max_path_Curve


In [8]:
dbgr = EpochHeuristicDebugger(p_x_given_n_masked=deepcopy(p_x_given_n_masked))
dbgr.build_ui()

slider = widgets.IntSlider(value=12, min=0, max=(a_decoder_decoded_epochs_result.num_filter_epochs-1))
slider.observe(dbgr.on_slider_change, names='value')
display(slider)

img_origin: (0.0, 0.0)
img_scale: (1.0, 1.0)


IntSlider(value=12, max=411)

epoch[13]
uniform_diffusion_prob: 0.017543859649122806
heuristic_scores: {'longest_sequence_length': 2, 'longest_sequence_length_ratio': 0.5, 'direction_change_bin_ratio': 0.6666666666666666, 'congruent_dir_bins_ratio': 0.6666666666666666, 'total_congruent_direction_change': 125.57876484467312, 'total_variation': 239.7412783398305, 'integral_second_derivative': 110.3570963786521, 'stddev_of_diff': 94.80505144727368}
epoch[0]
uniform_diffusion_prob: 0.017543859649122806
heuristic_scores: {'longest_sequence_length': 2, 'longest_sequence_length_ratio': 1.0, 'direction_change_bin_ratio': 0.0, 'congruent_dir_bins_ratio': 1.0, 'total_congruent_direction_change': 87.52459367962064, 'total_variation': 87.52459367962064, 'integral_second_derivative': 0.0, 'stddev_of_diff': 0.0}
epoch[1]
uniform_diffusion_prob: 0.017543859649122806
heuristic_scores: {'longest_sequence_length': 3, 'longest_sequence_length_ratio': 0.3, 'direction_change_bin_ratio': 0.5555555555555556, 'congruent_dir_bins_ratio': 0

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.heuristic_replay_scoring import HeuristicScoresTuple, HeuristicReplayScoring

dbgr.heuristic_scores = HeuristicReplayScoring.compute_pho_heuristic_replay_scores(a_result=a_decoder_decoded_epochs_result, an_epoch_idx=dbgr.active_single_epoch_result.epoch_data_index, debug_print=False)

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.heuristic_replay_scoring import HeuristicScoresTuple, HeuristicReplayScoring

HeuristicReplayScoring.bin_wise_continuous_sequence_sort_score_fn(a_result=a_decoder_decoded_epochs_result, an_epoch_idx=dbgr.active_single_epoch_result.epoch_data_index, a_decoder_track_length=170.0, same_thresh=4)

In [ ]:
# _curve_x = time_bin_centers
_curve_x = np.arange(len(max_path)) + 0.5 # move forward by a half bin

_max_path_Curve = dbgr.plot.addCurve(x=_curve_x, y=max_path, color='r', symbol='s', legend='max_path', replace=True, )
_max_path_Curve


In [ ]:
# def _my_split_list_fn(lst, accum, i, v):



from pyphoplacecellanalysis.Analysis.Decoder.heuristic_replay_scoring import partition_subsequences_ignoring_same_positions

lst1 = [0, 3.80542, -3.80542, -19.0271, 0, -19.0271]
list_parts1, list_split_indicies1 = partition_subsequences_ignoring_same_positions(lst=lst1) # [[0, 3.80542, -3.80542, 0, -19.0271]]
list_parts1
list_split_indicies1

lst2 = [0, 3.80542, 5.0, -3.80542, -19.0271, 0, -19.0271]
list_parts2, list_split_indicies2 = partition_subsequences_ignoring_same_positions(lst=lst2) # [[0, 3.80542, -3.80542], [-19.0271, 0, -19.0271]]
list_parts2
list_split_indicies2

# lst1 = [0, 3.80542, -3.80542, -19.0271, 0, -19.0271]
# partition_subsequences(lst=lst1) # [[0, 3.80542, -3.80542, 0, -19.0271]]

# lst2 = [0, 3.80542, 5.0, -3.80542, -19.0271, 0, -19.0271]
# partition_subsequences(lst=lst2) # [[0, 3.80542, -3.80542], [-19.0271, 0, -19.0271]]


In [ ]:
np.ang

In [ ]:
from rdp import rdp

max_path = np.nanargmax(dbgr.p_x_given_n_masked, axis=0) # returns the x-bins that maximize the path
_curve_x = np.arange(len(max_path)) + 0.5 # move forward by a half bin
# _curve_y = np.arange(len(max_path)) + 0.5 # move forward by a half bin
# assert len(max_path) == len(time_bin_centers)
# _curve_x = time_bin_centers

# print(nfx)
points = np.column_stack([_curve_x, max_path])
print("points.shape:", points.shape)
points_after_rdp = rdp(points, epsilon=1)

# Graph
plt.figure(num='rdp trajectory fit', clear=True)
plt.plot(points_after_rdp[:, 0], points_after_rdp[:, 1], color="red", label="after RDP")
plt.plot(
    points[:, 0], points[:, 1], color="black", label="before", alpha=0.7, linewidth=1
)
plt.xlabel("time (s)")
plt.ylabel("Close")
plt.legend()
plt.show()

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.heuristic_replay_scoring import _compute_total_variation, _compute_integral_second_derivative, _compute_stddev_of_diff

max_path = np.nanargmax(dbgr.p_x_given_n_masked, axis=0) # returns the x-bins that maximize the path

_compute_total_variation(max_path)
_compute_integral_second_derivative(max_path)
_compute_stddev_of_diff(max_path)


In [ ]:
# change_angles = np.degrees(np.arctan2(np.diff(max_path), time_bin_size))
change_angles = np.degrees(np.arctan2(np.diff(max_path), 1))
change_angles




In [ ]:
import numpy
from silx.gui import qt
from silx.gui.fit import FitWidget
from silx.math.fit.functions import sum_gauss

# x = numpy.arange(2000).astype(numpy.float)
# constant_bg = 3.14

# # gaussian parameters: height, position, fwhm
# p = numpy.array([1000, 100., 30.0,
#                  500, 300., 25.,
#                  1700, 500., 35.,
#                  750, 700., 30.0,
#                  1234, 900., 29.5,
#                  302, 1100., 30.5,
#                  75, 1300., 75.])
# y = sum_gauss(x, *p) + constant_bg

x = _curve_x
y = max_path

# a = qt.QApplication([])

w = FitWidget()
w.setData(x=x, y=y)
w.show()

In [ ]:
## INPUTS: active_captured_single_epoch_result

In [ ]:
# *position_relative": mapped between the ends of the track, 0.0 to 1.0
most_likely_position_relative: NDArray = (np.squeeze(active_captured_single_epoch_result.most_likely_position_indicies) / float(active_captured_single_epoch_result.n_xbins-1))
most_likely_position_relative

plt.hlines([0], colors='k', xmin=active_captured_single_epoch_result.time_bin_edges[0], xmax=active_captured_single_epoch_result.time_bin_edges[-1])
plt.step(active_captured_single_epoch_result.time_bin_container.centers[1:], np.diff(most_likely_position_relative))
plt.scatter(active_captured_single_epoch_result.time_bin_container.centers, most_likely_position_relative, color='r')


In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.heuristic_replay_scoring import HeuristicReplayScoring

# HeuristicReplayScoring.bin_wise_track_coverage_score_fn(a_result=a_decoder_decoded_epochs_result, an_epoch_idx=active_captured_single_epoch_result.epoch_data_index, a_decoder_track_length=170.0)
_out_new_scores = HeuristicReplayScoring.compute_pho_heuristic_replay_scores(a_result=a_decoder_decoded_epochs_result, an_epoch_idx=active_captured_single_epoch_result.epoch_data_index, debug_print=True)
# np.diff(active_captured_single_epoch_result.most_likely_position_indicies)
_out_new_scores

# 2024-05-05 - `BasicBinnedImageRenderingWindow` to render posteriors

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.BinnedImageRenderingWindow import BasicBinnedImageRenderingWindow, LayoutScrollability

# a posterior:
xbin_labels = [str(v) for v in dbgr.xbin_centers]
a_posterior = dbgr.result.p_x_given_n_list[dbgr.active_epoch_idx]
active_time_window_centers = deepcopy(dbgr.result.time_window_centers[dbgr.active_epoch_idx])


out = BasicBinnedImageRenderingWindow(a_posterior, active_time_window_centers, xbin_labels, name='posterior', title=f"Posterior for epoch[{dbgr.active_epoch_idx}]", variable_label='Avg Velocity', scrollability_mode=LayoutScrollability.SCROLLABLE)
# out.add_data(row=1, col=0, matrix=active_eloy_analysis.pf_overlapDensity_2D, xbins=active_pf_2D_dt.xbin_labels, ybins=active_pf_2D_dt.ybin_labels, name='pf_overlapDensity', title='pf overlapDensity metric', variable_label='pf overlapDensity')
# out.add_data(row=2, col=0, matrix=active_pf_2D.ratemap.occupancy, xbins=active_pf_2D.xbin, ybins=active_pf_2D.ybin, name='occupancy_seconds', title='Seconds Occupancy', variable_label='seconds')

# ⌚ 2024-05-06 - Old Jupyter slider with PIL.Image

In [ ]:
def _get_posterior_images(p_x_given_n):
    """ previews some processed posterior images """
    from PIL import Image
    
    img_plot_kwargs = dict(desired_width=None, desired_height=400, skip_img_normalization=False)
    posterior_image = get_array_as_image(p_x_given_n, **img_plot_kwargs).transpose(Image.FLIP_TOP_BOTTOM)
    # posterior_image

    # Define 8x8 blur filter kernel
    # blur_kernel = np.ones((8, 8)) #/ 64
    # blur_kernel = np.ones((8, 1)) # smear only along the y-axis
    blur_kernel = np.ones((1, 3)) # smear only along the x-axis
    blur_kernel = blur_kernel / np.sum(blur_kernel)

    # Apply blur to a 2D matrix
    blurred_matrix = convolve2d(p_x_given_n, blur_kernel, mode='same', boundary='fill') # 'wrap'
    blurred_image = get_array_as_image(blurred_matrix, **img_plot_kwargs).transpose(Image.FLIP_TOP_BOTTOM)
    # blurred_image

    # Display them side by side
    # display(posterior_image, blurred_image)

    # Concatenate the images horizontally
    h_border = 4
    concatenated_image = Image.new('RGB', ((posterior_image.width + h_border + blurred_image.width), max(posterior_image.height, blurred_image.height)))
    concatenated_image.paste(posterior_image, (0, 0))
    concatenated_image.paste(blurred_image, ((posterior_image.width + h_border), 0))
    # Display the concatenated image
    display(concatenated_image)

    # display(widgets.HBox([display(posterior_image), display(blurred_image)]))

# _get_posterior_images(p_x_given_n=p_x_given_n)
_get_posterior_images(p_x_given_n=p_x_given_n_masked)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Image, clear_output

## Interactive update.

active_captured_single_epoch_result: SingleEpochDecodedResult = a_decoder_decoded_epochs_result.get_result_for_epoch(active_epoch_idx=12)

def on_slider_change(change):
    clear_output(wait=True)  # Clear previous output
    print("Slider value:", change.new)
    active_captured_single_epoch_result = a_decoder_decoded_epochs_result.get_result_for_epoch(active_epoch_idx=change.new)
    posterior_image = active_captured_single_epoch_result.get_posterior_as_image(desired_height=200, skip_img_normalization=False)
    # display(posterior_image)
    
    # Apply blur to a 2D matrix
    blurred_matrix = convolve2d(active_captured_single_epoch_result.p_x_given_n, blur_kernel, mode='same', boundary='fill') # 'wrap'
    blurred_image = get_array_as_image(blurred_matrix, desired_height=200)
    # display(widgets.HBox([Image(posterior_image), Image(blurred_image)]))
    display(posterior_image, blurred_image)

slider = widgets.IntSlider(value=12, min=0, max=(a_decoder_decoded_epochs_result.num_filter_epochs-1))
slider.observe(on_slider_change, names='value')
out = widgets.Output()
display(slider, out)
# display(slider)

# looking good epochs: 12, 146, 


# 2024-04-25 - Interactive Posterior Constructor

In [ ]:
import sys
from PyQt5.QtWidgets import (QApplication, QMainWindow, QWidget, QVBoxLayout, 
                             QPushButton, QLabel, QGridLayout, QMessageBox)
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QPainter, QColor, QBrush

class WeightPainter(QMainWindow):
    def __init__(self, n=5, m=5):
        super().__init__()
        self.n = n  # Rows
        self.m = m  # Columns
        self.weights = [[0.0 for _ in range(self.m)] for _ in range(self.n)]
        self.current_tool = None
        self.init_ui()

    def init_ui(self):
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)

        self.layout = QVBoxLayout()
        self.buttons_layout = QGridLayout()
        self.grid_layout = QGridLayout()
        self.layout.addLayout(self.buttons_layout)
        self.layout.addLayout(self.grid_layout)
        self.central_widget.setLayout(self.layout)

        # Add buttons
        self.paint_button = QPushButton('Paint Weights', self)
        self.erase_button = QPushButton('Erase Weights', self)
        self.buttons_layout.addWidget(self.paint_button, 0, 0)
        self.buttons_layout.addWidget(self.erase_button, 0, 1)

        # Connect buttons to methods
        self.paint_button.clicked.connect(lambda: self.select_tool('paint'))
        self.erase_button.clicked.connect(lambda: self.select_tool('erase'))

        # Create labels for the matrix
        self.labels = [[QLabel('0.00', self) for _ in range(self.m)] for _ in range(self.n)]
        for i in range(self.n):
            for j in range(self.m):
                label = self.labels[i][j]
                label.setStyleSheet("QLabel { background-color: white; }")
                label.setAlignment(Qt.AlignCenter)
                self.grid_layout.addWidget(label, i, j)

        # Click event
        for row in self.labels:
            for label in row:
                label.mousePressEvent = self.cell_clicked

        # Window settings
        self.setGeometry(300, 300, 350, 250)
        self.setWindowTitle('Weight Painter')
        self.show()

    def select_tool(self, tool):
        self.current_tool = tool
        if tool == 'paint':
            self.paint_button.setEnabled(False)
            self.erase_button.setEnabled(True)
        elif tool == 'erase':
            self.paint_button.setEnabled(True)
            self.erase_button.setEnabled(False)

    def cell_clicked(self, event):
        if self.current_tool is None:
            QMessageBox.information(self, 'No tool selected',
                                    "Please select a tool before editing the weights.")
            return
        
        label = event.widget()
        i, j = self.get_label_position(label)
        if self.current_tool == 'paint':
            self.weights[i][j] += 0.1  # Increment weight
        elif self.current_tool == 'erase':
            self.weights[i][j] -= 0.1  # Decrement weight
            if self.weights[i][j] < 0:
                self.weights[i][j] = 0  # Prevent negative weights

        self.renormalize_column(j)
        self.update_labels()

    def get_label_position(self, label):
        for i, row in enumerate(self.labels):
            if label in row:
                return i, row.index(label)

    def renormalize_column(self, column):
        column_sum = sum(self.weights[i][column] for i in range(self.n))
        if column_sum == 0:
            return  # Avoid division by zero
        for i in range(self.n):
            self.weights[i][column] /= column_sum

    def update_labels(self):
        for i in range(self.n):
            for j in range(self.m):
                self.labels[i][j].setText(f"{self.weights[i][j]:.2f}")

# def main():
#     app = QApplication(sys.argv)
#     ex = WeightPainter()
#     sys.exit(app.exec_())

# if __name__ == '__main__':
#     main()
                
ex = WeightPainter()
ex.show()
ex

In [ ]:
ex.show()